#### Cartpole Game


In [3]:
import gym

In [4]:
env = gym.make('CartPole-v0')
env.reset()

array([ 0.03055808, -0.03097255,  0.0419786 , -0.04213528])

In [9]:
env.step(1)

(array([ 0.02801066, -0.03314685,  0.04014212,  0.00587138]), 1.0, False, {})

In [10]:
action = 1
next_state, reward, done, info = env.step(action)


#### Implementing Simple Neural Network using Keras

In [15]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [16]:

EPISODES = 1000

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        states, targets_f = [], []
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target 
            # Filtering out states and targets for training
            states.append(state[0])
            targets_f.append(target_f[0])
        history = self.model.fit(np.array(states), np.array(targets_f), epochs=1, verbose=0)
        # Keeping track of loss
        loss = history.history['loss'][0]
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        return loss

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


In [17]:
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
# agent.load("./save/cartpole-dqn.h5")
done = False
batch_size = 32

for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        # env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, EPISODES, time, agent.epsilon))
            break
        if len(agent.memory) > batch_size:
            loss = agent.replay(batch_size)
            # Logging training loss every 10 timesteps
            if time % 10 == 0:
                print("episode: {}/{}, time: {}, loss: {:.4f}"
                    .format(e, EPISODES, time, loss))  

episode: 0/1000, score: 11, e: 1.0
episode: 1/1000, score: 18, e: 1.0
episode: 2/1000, time: 10, loss: 2.1242
episode: 2/1000, time: 20, loss: 3.7592
episode: 2/1000, time: 30, loss: 2.5179
episode: 2/1000, score: 32, e: 0.86
episode: 3/1000, time: 0, loss: 0.9016
episode: 3/1000, time: 10, loss: 6.0206
episode: 3/1000, time: 20, loss: 2.2554
episode: 3/1000, score: 23, e: 0.76
episode: 4/1000, time: 0, loss: 2.2997
episode: 4/1000, score: 10, e: 0.73
episode: 5/1000, time: 0, loss: 4.3953
episode: 5/1000, score: 10, e: 0.69
episode: 6/1000, time: 0, loss: 7.9856
episode: 6/1000, time: 10, loss: 2.3921
episode: 6/1000, time: 20, loss: 0.6520
episode: 6/1000, time: 30, loss: 6.9581
episode: 6/1000, time: 40, loss: 4.7511
episode: 6/1000, time: 50, loss: 2.5397
episode: 6/1000, score: 52, e: 0.53
episode: 7/1000, time: 0, loss: 0.6888
episode: 7/1000, time: 10, loss: 2.8623
episode: 7/1000, time: 20, loss: 0.6236
episode: 7/1000, score: 22, e: 0.48
episode: 8/1000, time: 0, loss: 5.0641


episode: 61/1000, time: 160, loss: 15.0056
episode: 61/1000, time: 170, loss: 1.1075
episode: 61/1000, time: 180, loss: 6.9777
episode: 61/1000, score: 186, e: 0.01
episode: 62/1000, time: 0, loss: 4.8252
episode: 62/1000, time: 10, loss: 3.1343
episode: 62/1000, time: 20, loss: 2.2902
episode: 62/1000, time: 30, loss: 5.1868
episode: 62/1000, time: 40, loss: 0.5408
episode: 62/1000, time: 50, loss: 3.1284
episode: 62/1000, time: 60, loss: 6.7945
episode: 62/1000, time: 70, loss: 6.4456
episode: 62/1000, time: 80, loss: 0.8579
episode: 62/1000, time: 90, loss: 0.7381
episode: 62/1000, time: 100, loss: 2.1528
episode: 62/1000, time: 110, loss: 14.5644
episode: 62/1000, time: 120, loss: 1.4485
episode: 62/1000, score: 121, e: 0.01
episode: 63/1000, time: 0, loss: 1.5129
episode: 63/1000, time: 10, loss: 0.5273
episode: 63/1000, time: 20, loss: 0.7499
episode: 63/1000, time: 30, loss: 0.3780
episode: 63/1000, time: 40, loss: 3.7709
episode: 63/1000, time: 50, loss: 0.7702
episode: 63/1000

episode: 74/1000, time: 40, loss: 0.7064
episode: 74/1000, time: 50, loss: 0.1202
episode: 74/1000, time: 60, loss: 0.0338
episode: 74/1000, time: 70, loss: 0.0670
episode: 74/1000, time: 80, loss: 0.0857
episode: 74/1000, time: 90, loss: 0.2352
episode: 74/1000, time: 100, loss: 4.0240
episode: 74/1000, time: 110, loss: 0.0378
episode: 74/1000, time: 120, loss: 0.1101
episode: 74/1000, time: 130, loss: 0.0667
episode: 74/1000, time: 140, loss: 0.0738
episode: 74/1000, time: 150, loss: 0.0485
episode: 74/1000, time: 160, loss: 0.0508
episode: 74/1000, time: 170, loss: 0.0344
episode: 74/1000, time: 180, loss: 0.0364
episode: 74/1000, time: 190, loss: 0.0538
episode: 74/1000, time: 200, loss: 0.0476
episode: 74/1000, time: 210, loss: 0.0332
episode: 74/1000, time: 220, loss: 0.0207
episode: 74/1000, time: 230, loss: 0.0653
episode: 74/1000, time: 240, loss: 0.0341
episode: 74/1000, time: 250, loss: 0.0406
episode: 74/1000, time: 260, loss: 0.0922
episode: 74/1000, time: 270, loss: 0.307

episode: 79/1000, time: 230, loss: 2.5523
episode: 79/1000, time: 240, loss: 0.0318
episode: 79/1000, time: 250, loss: 0.0562
episode: 79/1000, time: 260, loss: 0.9577
episode: 79/1000, time: 270, loss: 0.0253
episode: 79/1000, time: 280, loss: 0.0715
episode: 79/1000, time: 290, loss: 0.0224
episode: 79/1000, time: 300, loss: 0.0225
episode: 79/1000, time: 310, loss: 0.0655
episode: 79/1000, time: 320, loss: 0.0159
episode: 79/1000, score: 326, e: 0.01
episode: 80/1000, time: 0, loss: 0.0254
episode: 80/1000, time: 10, loss: 0.0260
episode: 80/1000, time: 20, loss: 0.0113
episode: 80/1000, time: 30, loss: 0.0123
episode: 80/1000, time: 40, loss: 0.0120
episode: 80/1000, time: 50, loss: 0.0477
episode: 80/1000, time: 60, loss: 0.0260
episode: 80/1000, time: 70, loss: 0.0143
episode: 80/1000, time: 80, loss: 0.0282
episode: 80/1000, time: 90, loss: 0.0809
episode: 80/1000, time: 100, loss: 0.0481
episode: 80/1000, time: 110, loss: 0.0136
episode: 80/1000, time: 120, loss: 0.0390
episode

episode: 86/1000, time: 140, loss: 0.0168
episode: 86/1000, time: 150, loss: 0.0661
episode: 86/1000, time: 160, loss: 0.9373
episode: 86/1000, time: 170, loss: 0.0546
episode: 86/1000, time: 180, loss: 0.0549
episode: 86/1000, time: 190, loss: 0.0437
episode: 86/1000, score: 196, e: 0.01
episode: 87/1000, time: 0, loss: 0.4520
episode: 87/1000, time: 10, loss: 0.0141
episode: 87/1000, time: 20, loss: 0.0134
episode: 87/1000, time: 30, loss: 0.0176
episode: 87/1000, time: 40, loss: 0.0094
episode: 87/1000, time: 50, loss: 0.0469
episode: 87/1000, time: 60, loss: 0.0232
episode: 87/1000, time: 70, loss: 0.0636
episode: 87/1000, time: 80, loss: 1.3171
episode: 87/1000, time: 90, loss: 0.0240
episode: 87/1000, time: 100, loss: 0.0157
episode: 87/1000, time: 110, loss: 0.0179
episode: 87/1000, time: 120, loss: 0.8345
episode: 87/1000, time: 130, loss: 0.1097
episode: 87/1000, time: 140, loss: 0.0148
episode: 87/1000, time: 150, loss: 0.0306
episode: 87/1000, time: 160, loss: 0.0161
episode

...

episode: 959/1000, time: 50, loss: 0.0846
episode: 959/1000, time: 60, loss: 0.0164
episode: 959/1000, time: 70, loss: 0.0148
episode: 959/1000, time: 80, loss: 0.0411
episode: 959/1000, time: 90, loss: 0.0248
episode: 959/1000, time: 100, loss: 0.0157
episode: 959/1000, time: 110, loss: 0.0189
episode: 959/1000, time: 120, loss: 0.0176
episode: 959/1000, time: 130, loss: 0.0260
episode: 959/1000, time: 140, loss: 0.0269
episode: 959/1000, time: 150, loss: 0.0215
episode: 959/1000, time: 160, loss: 0.0178
episode: 959/1000, time: 170, loss: 0.2141
episode: 959/1000, time: 180, loss: 0.0187
episode: 959/1000, time: 190, loss: 0.0128
episode: 959/1000, time: 200, loss: 0.0142
episode: 959/1000, time: 210, loss: 0.0059
episode: 959/1000, time: 220, loss: 0.1187
episode: 959/1000, score: 229, e: 0.01
episode: 960/1000, time: 0, loss: 0.0107
episode: 960/1000, time: 10, loss: 0.0127
episode: 960/1000, time: 20, loss: 0.0161
episode: 960/1000, time: 30, loss: 0.1761
episode: 960/1000, time: 

episode: 967/1000, time: 120, loss: 0.0159
episode: 967/1000, time: 130, loss: 0.1527
episode: 967/1000, time: 140, loss: 0.0263
episode: 967/1000, time: 150, loss: 0.0504
episode: 967/1000, time: 160, loss: 0.0156
episode: 967/1000, time: 170, loss: 0.0158
episode: 967/1000, time: 180, loss: 0.0156
episode: 967/1000, time: 190, loss: 0.0104
episode: 967/1000, time: 200, loss: 0.0262
episode: 967/1000, score: 209, e: 0.01
episode: 968/1000, time: 0, loss: 0.0226
episode: 968/1000, time: 10, loss: 0.0530
episode: 968/1000, time: 20, loss: 0.0109
episode: 968/1000, time: 30, loss: 0.3745
episode: 968/1000, time: 40, loss: 0.1817
episode: 968/1000, time: 50, loss: 0.0103
episode: 968/1000, time: 60, loss: 0.0053
episode: 968/1000, time: 70, loss: 0.0059
episode: 968/1000, time: 80, loss: 0.0087
episode: 968/1000, time: 90, loss: 0.0120
episode: 968/1000, time: 100, loss: 0.0047
episode: 968/1000, time: 110, loss: 0.0167
episode: 968/1000, time: 120, loss: 0.0126
episode: 968/1000, time: 1

episode: 976/1000, time: 160, loss: 0.0046
episode: 976/1000, time: 170, loss: 0.0051
episode: 976/1000, time: 180, loss: 0.0110
episode: 976/1000, score: 184, e: 0.01
episode: 977/1000, time: 0, loss: 0.0139
episode: 977/1000, time: 10, loss: 0.0149
episode: 977/1000, time: 20, loss: 0.0066
episode: 977/1000, time: 30, loss: 0.1743
episode: 977/1000, time: 40, loss: 0.0270
episode: 977/1000, time: 50, loss: 0.0123
episode: 977/1000, time: 60, loss: 0.2417
episode: 977/1000, time: 70, loss: 0.0107
episode: 977/1000, time: 80, loss: 0.0044
episode: 977/1000, time: 90, loss: 0.0055
episode: 977/1000, time: 100, loss: 0.0031
episode: 977/1000, time: 110, loss: 0.0046
episode: 977/1000, time: 120, loss: 0.0047
episode: 977/1000, time: 130, loss: 0.0045
episode: 977/1000, time: 140, loss: 0.0105
episode: 977/1000, time: 150, loss: 0.4213
episode: 977/1000, time: 160, loss: 0.2195
episode: 977/1000, time: 170, loss: 0.0154
episode: 977/1000, time: 180, loss: 0.0113
episode: 977/1000, time: 1

episode: 985/1000, time: 260, loss: 0.0020
episode: 985/1000, time: 270, loss: 0.0471
episode: 985/1000, time: 280, loss: 1.2711
episode: 985/1000, time: 290, loss: 0.0094
episode: 985/1000, time: 300, loss: 0.0119
episode: 985/1000, time: 310, loss: 0.0044
episode: 985/1000, score: 311, e: 0.01
episode: 986/1000, time: 0, loss: 0.0222
episode: 986/1000, time: 10, loss: 0.0095
episode: 986/1000, time: 20, loss: 0.0059
episode: 986/1000, time: 30, loss: 0.0627
episode: 986/1000, time: 40, loss: 0.0105
episode: 986/1000, time: 50, loss: 0.2711
episode: 986/1000, time: 60, loss: 0.0237
episode: 986/1000, time: 70, loss: 0.0497
episode: 986/1000, time: 80, loss: 0.0254
episode: 986/1000, time: 90, loss: 0.0181
episode: 986/1000, time: 100, loss: 0.0093
episode: 986/1000, time: 110, loss: 0.0198
episode: 986/1000, time: 120, loss: 0.6909
episode: 986/1000, time: 130, loss: 0.0075
episode: 986/1000, time: 140, loss: 0.0088
episode: 986/1000, time: 150, loss: 0.0020
episode: 986/1000, time: 1

episode: 992/1000, time: 140, loss: 0.1120
episode: 992/1000, time: 150, loss: 0.0324
episode: 992/1000, time: 160, loss: 0.0333
episode: 992/1000, time: 170, loss: 0.0263
episode: 992/1000, time: 180, loss: 0.1237
episode: 992/1000, time: 190, loss: 0.3632
episode: 992/1000, time: 200, loss: 0.0471
episode: 992/1000, time: 210, loss: 0.0445
episode: 992/1000, time: 220, loss: 0.1621
episode: 992/1000, time: 230, loss: 0.0436
episode: 992/1000, time: 240, loss: 0.0827
episode: 992/1000, score: 249, e: 0.01
episode: 993/1000, time: 0, loss: 0.0282
episode: 993/1000, time: 10, loss: 0.0680
episode: 993/1000, time: 20, loss: 0.0466
episode: 993/1000, time: 30, loss: 0.0518
episode: 993/1000, time: 40, loss: 0.0478
episode: 993/1000, time: 50, loss: 0.4814
episode: 993/1000, time: 60, loss: 0.1414
episode: 993/1000, time: 70, loss: 0.1326
episode: 993/1000, time: 80, loss: 0.7031
episode: 993/1000, time: 90, loss: 0.1672
episode: 993/1000, time: 100, loss: 0.1101
episode: 993/1000, time: 1